In [1]:
from chessdotcom import get_player_profile, get_player_stats, get_player_game_archives
import requests
import pprint
import pandas as pd
from parsita import *
from parsita.util import constant
import json
import chess.pgn
import io
import ftplib
import os
import logging

# https://pypi.org/project/pgn2data/
from converter.pgn_data import PGNData

In [2]:
"""
chess_fact table
player_id
game_id
move_id

dimention tables

Player_table
Player id - primary key
player username
class
player rating
rating updated time


Game_table
game id - primary key
player id - foreign key
urls
time_control
Date
EndDate
StartTime
EndTime
Timezone
UTCDate
UTCTime
initial setup
time class
rules
white_rating
white_username
black_rating
black_username
player_rating
event
Site
Round
Result
CurrentPosition
ECO
ECOUrl
Termination
pgn

Moves_table
game_id
move_id
move_number
white_move
black_move
white_time
black_time
"""

'\nchess_fact table\nplayer_id\ngame_id\nmove_id\n\ndimention tables\n\nPlayer_table\nPlayer id - primary key\nplayer username\nclass\nplayer rating\nrating updated time\n\n\nGame_table\ngame id - primary key\nplayer id - foreign key\nurls\ntime_control\nDate\nEndDate\nStartTime\nEndTime\nTimezone\nUTCDate\nUTCTime\ninitial setup\ntime class\nrules\nwhite_rating\nwhite_username\nblack_rating\nblack_username\nplayer_rating\nevent\nSite\nRound\nResult\nCurrentPosition\nECO\nECOUrl\nTermination\npgn\n\nMoves_table\ngame_id\nmove_id\nmove_number\nwhite_move\nblack_move\nwhite_time\nblack_time\n'

In [3]:
# Fetch data from chess.com API

# make json data easier to read
pp = pprint.PrettyPrinter(width=41, compact=True)

def get_user_archives(username, months):
    """
    get archive monthly files of specific chess.com player
    input:
    username - username of the chess.com player
    months - target months that we want to get the archives
    
    output:
    target_month - files of archives according to months parameter
    """
    archives = get_player_game_archives(username).json['archives']
    target_month = []
    for archive in archives:
        if archive[-7:] in months:
            target_month.append(archive)
    return target_month

#files = get_user_archives("tianminlyu",["2023/01"])


In [4]:
def get_archive_games(filename):
    """
    return games in one archive file
    
    input:
    filename - filename that contains game urls
    
    output: 
    """
    games = requests.get(filename).json()['games']
    return games

#games = get_archive_games(files[-1])

In [5]:
def game_df(username,files):
    """
    import data from archive files and turn relevant data parameters into data frames
    
    input:
    username - username of the player
    files - file archived of the player
    
    output:
    a dataframe contains wanted columns
                        'username',
                         'urls', 
                         'time_control',
                        'end_time',
                        'uuid',
                        'initial_setup',
                        'time_class',
                        'rules',
                        'white_rating',
                        'white_username',
                        'black_rating',
                        'black_username',
                        'Event',
                        'Site',
                        'Date',
                        'Round',
                        'Result',
                        'CurrentPosition',
                        'ECO',
                        'ECOUrl',
                        'EndDate',
                        'EndTime',
                        'StartTime',
                        'Termination',
                        'Timezone',
                        'UTCDate',
                        'UTCTime',                        
                        
    """
    print("Player " + username + " is processing...")
    usernames = []
    urls = []
    time_control = [] 
    end_time = [] 
    uuid = []
    initial_setup = [] 
    time_class = [] 
    rules = [] 
    white_rating = [] 
    white_username = [] 
    black_rating = [] 
    black_username = []
    pgn = []
    event = []
    Site = []
    Date = []
    Round = []
    Result = []
    CurrentPosition = []
    ECO = []
    ECOUrl = []
    EndDate = []
    EndTime = []
    StartTime = []
    Termination = []
    Timezone = []
    UTCDate = []
    UTCTime = []
    
    for file in files:
        print(file + " " + " is processing...")
        games = get_archive_games(file)
        for game in games:
            try:
                usernames.append(username)
                urls.append(game['url'])
                time_control.append(game['time_control'])
                end_time.append(game['end_time'])
                uuid.append(game['uuid'])
                initial_setup.append(game['initial_setup'])
                time_class.append(game['time_class'])
                rules.append(game['rules'])
                white_rating.append(game['white']['rating'])
                white_username.append(game['white']['username'])
                black_rating.append(game['black']['rating'])
                black_username.append(game['black']['username'])
                pgn.append(game['pgn'])
                
                pgn_written = io.StringIO(game['pgn'])
                game_data = chess.pgn.read_game(pgn_written)
                event.append(game_data.headers['Event'])
                Site.append(game_data.headers['Site'])
                Date.append(game_data.headers['Date'])
                Round.append(game_data.headers['Round'])
                Result.append(game_data.headers['Result'])
                CurrentPosition.append(game_data.headers['CurrentPosition'])
                ECO.append(game_data.headers['ECO'])
                ECOUrl.append(game_data.headers['ECOUrl'])
                EndDate.append(game_data.headers['EndDate'])
                EndTime.append(game_data.headers['EndTime'])
                StartTime.append(game_data.headers['StartTime'])
                Termination.append(game_data.headers['Termination'])
                Timezone.append(game_data.headers['Timezone'])
                UTCDate.append(game_data.headers['UTCDate'])
                UTCTime.append(game_data.headers['UTCTime'])
            except Exception as e:
                print(e)
                print(game['url'])
    
    print("data fetch work is done.")
    
    df = pd.DataFrame(list(zip(usernames,
                               urls, 
                           time_control,
                          end_time,
                           uuid,
                           initial_setup,
                           time_class,
                           rules,
                           white_rating,
                           white_username,
                           black_rating,
                           black_username,
                           pgn,
                           event,
                          Site,
                          Date,
                          Round,
                          Result,
                          CurrentPosition,
                          ECO,
                          ECOUrl,
                          EndDate,
                          EndTime,
                          StartTime,
                          Termination,
                          Timezone,
                          UTCDate,
                          UTCTime)),
               columns =['username',
                         'urls', 
                         'time_control',
                        'end_time',
                        'uuid',
                        'initial_setup',
                        'time_class',
                        'rules',
                        'white_rating',
                        'white_username',
                        'black_rating',
                        'black_username',
                         'pgn',
                         'event',
                         'Site',
                          'Date',
                          'Round',
                          'Result',
                          'CurrentPosition',
                          'ECO',
                          'ECOUrl',
                          'EndDate',
                          'EndTime',
                          'StartTime',
                          'Termination',
                          'Timezone',
                          'UTCDate',
                          'UTCTime'
                        ])
    print("dataframe importing is done.")
    return df
    
#data = game_df('tianminlyu',files)


In [6]:
# students from Tianmin's classes - BO, BP, AN
tianmin_players = {
    "BO" : ['AGcuber19',
            'TLPAWN',
            'xiaoanwu',
            'EmmaXLi',
            'akfunchess66',
            'Marsboom', 
            'Claraqiu',
            'Ravenclawfairy', 
            'Zora_zhu',
            'BurleyWalrus'],
    "BP" : ['taionemm',
            'augustinewz',
            'oscarzhang818',
            'yaohengli',
            'Wallacewang1214',
            'SophiaZ2022',
            'AliceCLi',
            'yumitang',
            'james2945',
            'Oinkoinkw'],
    "AN" : ['Cathye1',
            'lunathekitsune',
            'ArthurRocket',
            'vivianwwww20',
            'ChloeWang16',
            'Tyzalex',
            'ZhichengW',
            'Haochen1123',
            'jaydenlan0118',
            'ImRacoonie']
}

In [7]:
df_players = []
error_players = []

for classes in tianmin_players.keys():
    for player in tianmin_players[classes]:
        try:
            files = get_user_archives(player,["2023/02","2023/01","2022/12"]) # target months
            df = game_df(player, files)
            df['class'] = classes
            df_players.append(df)
        except:
            print(Exception)
            print("This player account " + player + " does not exist")
            error_players.append(player)

Player AGcuber19 is processing...
https://api.chess.com/pub/player/agcuber19/games/2022/12  is processing...
'ECO'
https://www.chess.com/game/live/63766207495
'ECO'
https://www.chess.com/game/live/64002753283
'ECO'
https://www.chess.com/game/live/64089271325
https://api.chess.com/pub/player/agcuber19/games/2023/01  is processing...
'ECO'
https://www.chess.com/game/live/67449082651
https://api.chess.com/pub/player/agcuber19/games/2023/02  is processing...
'ECO'
https://www.chess.com/game/live/69024314043
'pgn'
https://www.chess.com/game/live/69027164491
'ECO'
https://www.chess.com/game/live/69027243575
data fetch work is done.
dataframe importing is done.
Player TLPAWN is processing...
https://api.chess.com/pub/player/tlpawn/games/2022/12  is processing...
'ECO'
https://www.chess.com/game/daily/456493725
https://api.chess.com/pub/player/tlpawn/games/2023/01  is processing...
'ECO'
https://www.chess.com/game/live/67387278311
https://api.chess.com/pub/player/tlpawn/games/2023/02  is proce

data fetch work is done.
dataframe importing is done.
Player ZhichengW is processing...
https://api.chess.com/pub/player/zhichengw/games/2023/01  is processing...
https://api.chess.com/pub/player/zhichengw/games/2023/02  is processing...
data fetch work is done.
dataframe importing is done.
Player Haochen1123 is processing...
https://api.chess.com/pub/player/haochen1123/games/2023/01  is processing...
https://api.chess.com/pub/player/haochen1123/games/2023/02  is processing...
data fetch work is done.
dataframe importing is done.
Player jaydenlan0118 is processing...
https://api.chess.com/pub/player/jaydenlan0118/games/2023/01  is processing...
https://api.chess.com/pub/player/jaydenlan0118/games/2023/02  is processing...
data fetch work is done.
dataframe importing is done.
Player ImRacoonie is processing...
https://api.chess.com/pub/player/imracoonie/games/2023/01  is processing...
https://api.chess.com/pub/player/imracoonie/games/2023/02  is processing...
data fetch work is done.
da

In [8]:
print("% of error players is ...")
print(len(error_players) * 100/ len(tianmin_players))

% of error players is ...
0.0


In [9]:
players_df = pd.concat(df_players)

In [10]:
def player_rating(row):  
    if row['username'].lower() == row['white_username'].lower():
        return row['white_rating']
    else:
        return row['black_rating']
    
players_df['player_rating'] = players_df.apply(lambda row: player_rating(row), axis=1)

In [11]:
# generate moves table from pgn column

def moves_split(pgn):
    move_number = []
    white_move = []
    black_move = []
    white_time = []
    black_time = []
    for note in range(len(pgn)):
        if note % 8 == 0:
            move_number.append(pgn[note].split(".")[0])
        elif note % 8 == 1:
            white_move.append(pgn[note])
        elif note % 8 == 2:
            pass
        elif note % 8== 3:
            white_time.append(pgn[note].split("]}")[0])
        elif note %8 == 4:
            pass
        elif note % 8 == 5:
            black_move.append(pgn[note])
        elif note % 8 == 6:
            pass
        elif note % 8 == 7:
            black_time.append(pgn[note].split("]}")[0])
        
    if len(black_move) < len(white_move):
        black_move.append("NaN")
        black_time.append("NaN")

    return move_number, white_move, black_move, white_time, black_time

In [12]:
def create_moves_df(game):
    moves_df = []
    for i in range(game.shape[0]):
        pgn = game['pgn'].iloc[i].split("\n")[-2].split(" ")[:-1]
        moves = moves_split(pgn)
        move_number = moves[0]
        white_move = moves[1]
        black_move = moves[2]
        white_time = moves[3]
        black_time = moves[4]
        uuid = [game['uuid'].iloc[i]] * len(move_number)


        df = pd.DataFrame(list(zip(uuid,
                              move_number,
                              white_move,
                              black_move,
                              white_time,
                              black_time)),
               columns =[     'uuid',
                              'move_number',
                              'white_move',
                              'black_move',
                              'white_time',
                              'black_time'
                        ])
        moves_df.append(df)
    moves_df = pd.concat(moves_df)
    return moves_df

In [13]:
players_df.head()

,username,urls,time_control,end_time,uuid,initial_setup,time_class,rules,white_rating,white_username,...,ECOUrl,EndDate,EndTime,StartTime,Termination,Timezone,UTCDate,UTCTime,class,player_rating
0,AGcuber19,https://www.chess.com/game/live/63764962677,600,1670026173,efe6de36-729d-11ed-a69d-78ac4409ff3c,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,rapid,chess,739,nien-eleven,...,https://www.chess.com/openings/Englund-Gambit-...,2022.12.03,00:09:33,00:03:42,nien-eleven won by resignation,UTC,2022.12.03,00:03:42,BO,640
1,AGcuber19,https://www.chess.com/game/live/63765037219,600,1670027162,c44f652e-729e-11ed-a69d-78ac4409ff3c,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,rapid,chess,602,Sebastianbulowthomsen,...,https://www.chess.com/openings/Kings-Pawn-Open...,2022.12.03,00:26:02,00:09:35,Sebastianbulowthomsen won by checkmate,UTC,2022.12.03,00:09:35,BO,631
2,AGcuber19,https://www.chess.com/game/live/63766207495,600,1670027246,3fc79932-72a1-11ed-a69d-78ac4409ff3c,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,rapid,chess,605,xfoddz,...,https://www.chess.com/openings/Nimzowitsch-Def...,2022.12.03,00:39:02,00:27:28,Eric_Carstens won by resignation,UTC,2022.12.03,00:27:28,BO,639
3,AGcuber19,https://www.chess.com/game/live/63766210035,600,1670027942,46e3ab94-72a1-11ed-a69d-78ac4409ff3c,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,rapid,chess,631,AGcuber19,...,https://www.chess.com/openings/Kings-Pawn-Open...,2022.12.03,00:40:45,00:39:07,AGcuber19 won - game abandoned,UTC,2022.12.03,00:39:07,BO,631
4,AGcuber19,https://www.chess.com/game/live/63766827497,600,1670028045,e39207bd-72a2-11ed-a69d-78ac4409ff3c,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,rapid,chess,637,AGcuber19,...,https://www.chess.com/openings/Kings-Pawn-Open...,2022.12.03,00:51:06,00:41:26,AGcuber19 won - game abandoned,UTC,2022.12.03,00:41:26,BO,637


In [14]:
moves = create_moves_df(players_df)

In [15]:
moves

,uuid,move_number,white_move,black_move,white_time,black_time
0,efe6de36-729d-11ed-a69d-78ac4409ff3c,1,d4,e5,0:10:00,0:09:57.4
1,efe6de36-729d-11ed-a69d-78ac4409ff3c,2,dxe5,Qh4,0:09:58.8,0:09:54.9
2,efe6de36-729d-11ed-a69d-78ac4409ff3c,3,Nf3,Qb4+,0:09:57.4,0:09:47.4
3,efe6de36-729d-11ed-a69d-78ac4409ff3c,4,c3,Qb6,0:09:52.9,0:09:31.2
4,efe6de36-729d-11ed-a69d-78ac4409ff3c,5,Be3,Qxb2,0:09:45.6,0:09:22.8
...,...,...,...,...,...,...
24,93c76789-a4b8-11ed-8eba-78ac4409ff3c,25,cxd5,Rxd5,0:15:36.1,0:14:42.7
25,93c76789-a4b8-11ed-8eba-78ac4409ff3c,26,Qc2,Rf5+,0:15:40.7,0:14:35.5
26,93c76789-a4b8-11ed-8eba-78ac4409ff3c,27,Kg2,Nf1+,0:15:43.5,0:14:33.1
27,93c76789-a4b8-11ed-8eba-78ac4409ff3c,28,Kh3,Rf6,0:15:48.6,0:14:39.9


In [16]:
# https://docs.google.com/spreadsheets/d/1QJjp2wG_k4XPW6Z8hbvArKkCQH3fI5O7RmPH2Ongto4/edit#gid=0

"""
chess_fact table
player_id
game_id
move_id
"""



'\nchess_fact table\nplayer_id\ngame_id\nmove_id\n'

In [17]:
"""
Player_table
Player id - primary key
player username
class
player rating
rating updated time
"""



'\nPlayer_table\nPlayer id - primary key\nplayer username\nclass\nplayer rating\nrating updated time\n'

In [18]:
"""
Game_table
game id - primary key
player id - foreign key?
urls
time_control
Date
EndDate
StartTime
EndTime
Timezone
UTCDate
UTCTime
initial setup
time class
rules
white_rating
white_username
black_rating
black_username
player_rating
event
Site
Round
Result
CurrentPosition
ECO
ECOUrl
Termination
pgn
"""

'\nGame_table\ngame id - primary key\nplayer id - foreign key?\nurls\ntime_control\nDate\nEndDate\nStartTime\nEndTime\nTimezone\nUTCDate\nUTCTime\ninitial setup\ntime class\nrules\nwhite_rating\nwhite_username\nblack_rating\nblack_username\nplayer_rating\nevent\nSite\nRound\nResult\nCurrentPosition\nECO\nECOUrl\nTermination\npgn\n'

In [19]:
"""
Moves_table
game_id
move_id
move_number
white_move
black_move
white_time
black_time
"""

'\nMoves_table\ngame_id\nmove_id\nmove_number\nwhite_move\nblack_move\nwhite_time\nblack_time\n'